# 01_EDA – Análisis Exploratorio de Datos

Este notebook corresponde a la fase **Data Understanding** del enfoque CRISP-DM, dentro del desafío de forecasting M+2.  
Aquí realizamos un análisis exploratorio profundo de los datos, buscando identificar patrones de ventas, características de productos, y comportamientos de clientes.  
Aplicamos enfoques **Top-Down** y **Bottom-Up** para asegurar una comprensión completa de la dinámica del negocio.

**Fuentes de datos analizadas**:
- `sell-in_muestra.txt`: ventas mensuales por cliente y producto.
- `tb_productos.txt`: maestro de productos.
- `tb_stocks.txt`: stock mensual por producto.


## Carga de librerías y rutas de trabajo

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ipywidgets import interact

# Rutas de archivos
DATA_PATH = r"C:\Developer\Laboratorio_III\data"
SELL_IN_FILE = DATA_PATH + r"\sell-in_muestra.txt"
PRODUCT_FILE = DATA_PATH + r"\tb_productos.txt"
STOCK_FILE = DATA_PATH + r"\tb_stocks.txt"


## Carga de datos y ajuste del campo `periodo` como texto

In [ ]:

def load_txt(path, **kwargs):
    return pd.read_csv(path, sep="\t", engine="python", **kwargs)

df_sales = load_txt(SELL_IN_FILE)
df_prod = load_txt(PRODUCT_FILE)
df_stock = load_txt(STOCK_FILE)

for df in [df_sales, df_stock]:
    df['periodo'] = df['periodo'].astype(str)

display(df_sales.head(), df_prod.head(), df_stock.head())


## Integración de datasets

In [ ]:

df_all = (
    df_sales
    .merge(df_prod, on='product_id', how='left')
    .merge(df_stock, on=['product_id', 'periodo'], how='left')
)

df_all = df_all.sort_values(['periodo', 'product_id'])
df_all.to_parquet(DATA_PATH + r"\dataset_integrado.parquet")
df_all.head()


## Análisis Top-Down: Tendencias por familia de productos
Buscamos patrones agregados por familia (`cat1`) a lo largo del tiempo para identificar estacionalidad, crecimientos o caídas.

In [ ]:

agg_family = df_all.groupby(['periodo', 'cat1'], observed=True)['tn'].sum().reset_index()
pivot = agg_family.pivot(index='periodo', columns='cat1', values='tn')
pivot.plot(figsize=(12, 4))
plt.title('Toneladas vendidas por familia')
plt.ylabel('tn')
plt.xlabel('Periodo (texto)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Análisis Bottom-Up: Clasificación ABC-XYZ
Evaluamos la importancia y la variabilidad de cada SKU para entender cuáles son estratégicos y estables.

### ¿Qué es la clasificación ABC?

La clasificación **ABC** es una técnica basada en el principio de Pareto, utilizada para segmentar productos (u otras entidades) según su importancia en el volumen total de negocio.  
Se basa en el porcentaje acumulado de volumen que representa cada producto:

- **A**: productos más importantes (∼80% del volumen total). Son críticos para el negocio.
- **B**: productos intermedios (∼15% del volumen).
- **C**: productos menos significativos (∼5% del volumen restante).

**Criterio aplicado:**  
Calculamos el total de toneladas (`tn`) por `product_id`, ordenamos de mayor a menor, y luego acumulamos el porcentaje que representa cada SKU sobre el total.  
A partir de ahí, segmentamos con cortes fijos:  
- A: 0–80%, B: 80–95%, C: 95–100%.


### ¿Qué es la clasificación XYZ?

La clasificación **XYZ** se basa en la **variabilidad de la demanda**, y busca responder qué tan regular o errática es la venta de cada producto.  
Se utiliza el **Coeficiente de Variación (CV)** para esta segmentación:

**Cálculo aplicado:**  
A partir de una tabla período × cliente (`pivot_clientes`), aplicamos:


$$
CV = \frac{\sigma}{\mu}
$$


- **X**: productos muy estables (CV < 0.5)
- **Y**: productos moderadamente variables (CV entre 0.5 y 1)
- **Z**: productos muy volátiles o erráticos (CV > 1)

**Interpretación general:**
- Productos **X** son predecibles y fáciles de modelar.
- Productos **Z** requieren atención especial o técnicas robustas.

**Nota:** El CV es sensible a la media. Si un producto tiene una media muy baja, el CV puede volverse muy grande o indefinido. En esos casos, descartamos o imputamos nulos.


In [ ]:

sku_tot = df_all.groupby('product_id')['tn'].sum().sort_values(ascending=False)
cum_pct = sku_tot.cumsum() / sku_tot.sum()
abc_class = pd.cut(cum_pct, bins=[0, .8, .95, 1], labels=list('ABC'))
cov = df_all.groupby('product_id')['tn'].apply(lambda x: x.std()/x.mean() if x.mean()!=0 else np.nan)
xyz_class = pd.cut(cov, bins=[-np.inf, .5, 1, np.inf], labels=list('XYZ'))

abc_df = pd.DataFrame({'total_tn': sku_tot, 'cum_pct': cum_pct, 'ABC': abc_class, 'XYZ': xyz_class})
sns.heatmap(pd.crosstab(abc_df['ABC'], abc_df['XYZ']), annot=True, fmt='d')
plt.title('Matriz ABC‑XYZ')
plt.show()


## Visualización: Volumen vs. Variabilidad de SKU
Usamos la misma clasificación ABC basada en acumulado de volumen para identificar SKU prioritarios para el modelado.

In [ ]:

df_cv = df_all.groupby('product_id')['tn'].apply(lambda x: x.std()/x.mean() if x.mean()!=0 else None)
df_stats = pd.concat([sku_tot.rename("total_tn"), df_cv.rename("cv"), abc_class.rename("ABC")], axis=1).dropna()

fig = px.scatter(df_stats, x='total_tn', y='cv', color='ABC',
                 hover_name=df_stats.index.astype(str),
                 log_x=True, height=500,
                 title='Dispersión: Volumen total vs. Volatilidad (CV)',
                 labels={'total_tn': 'Toneladas Totales (log)', 'cv': 'Coef. Variación'})
fig.show()


## Análisis de clientes (customer_id)
Estudiamos el comportamiento de compra por cliente para identificar perfiles estables, erráticos, y patrones de fidelidad.

In [ ]:

clientes_top = df_all.groupby('customer_id')['tn'].sum().sort_values(ascending=False).head(20)
clientes_top.plot(kind='bar', figsize=(10,4), color='teal')
plt.title('Top 20 clientes por toneladas compradas')
plt.ylabel('Toneladas')
plt.xlabel('ID Cliente')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

pivot_clientes = df_all.pivot_table(index='periodo', columns='customer_id', values='tn', aggfunc='sum', fill_value=0)

@interact(cliente=sorted(pivot_clientes.columns))
def grafico_cliente(cliente):
    pivot_clientes[cliente].plot(figsize=(10,3))
    plt.title(f'Evolución mensual de compras - Cliente {cliente}')
    plt.ylabel('Toneladas')
    plt.xlabel('Periodo')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


### Variabilidad en el comportamiento de clientes

También aplicamos el **Coeficiente de Variación (CV)** para analizar la estabilidad de las compras mensuales de cada cliente.  
Esto nos permite detectar perfiles de compra:

- Clientes estables (CV bajo): compran cantidades similares cada mes.
- Clientes erráticos (CV alto): compran de forma muy irregular.

**Cálculo aplicado:**  
A partir de una tabla período × cliente (`pivot_clientes`), aplicamos:

\[
CV_{cliente} = \frac{\text{Desvío estándar mensual}}{\text{Media mensual de compras}}
\]

Este análisis ayuda a entender a qué clientes se les puede aplicar modelos simples o si requieren un tratamiento más específico.


In [ ]:

cv_clientes = pivot_clientes.std() / pivot_clientes.mean()
cv_clientes = cv_clientes.replace([np.inf, -np.inf], np.nan).dropna()

cv_clientes.hist(bins=30, figsize=(6,3))
plt.title('Distribución del Coef. de Variación entre clientes')
plt.xlabel('CV (Desvío / Media)')
plt.ylabel('Cantidad de clientes')
plt.grid(True)
plt.tight_layout()
plt.show()


## Volumen vs. variabilidad en clientes

In [ ]:

volumen_clientes = pivot_clientes.sum()
df_clientes = pd.DataFrame({'total_tn': volumen_clientes, 'cv': cv_clientes}).dropna()

fig = px.scatter(df_clientes,
                 x='total_tn', y='cv',
                 hover_name=df_clientes.index.astype(str),
                 log_x=True,
                 labels={'total_tn': 'Toneladas totales (log)', 'cv': 'Coef. Variación'},
                 title='Dispersión: Volumen total vs. Variabilidad de compras (clientes)',
                 height=500)
fig.show()
